In [ ]:
df=pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv', encoding='latin')
df.head()

In [ ]:
df['attack_cat'].unique()

In [ ]:
(unique, counts) = np.unique(df['attack_cat'], return_counts=True)

print('Unique values of the target variable', unique)
print('Counts of the target variable :', counts)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
plt.rcParams.update({'font.size': 14})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
ax.bar(unique, counts)
plt.show()

In [ ]:
df.replace(to_replace=["Analysis"], value="Attacks_1", inplace=True)
df.replace(to_replace=["Backdoor"], value="Attacks_1", inplace=True)
df.replace(to_replace=["Shellcode"], value="Attacks_1", inplace=True)
df.replace(to_replace=["Worms"], value="Attacks_1", inplace=True)

In [ ]:
df = df[df['attack_cat'] != 'Attacks_1']

In [ ]:
(unique, counts) = np.unique(df['attack_cat'], return_counts=True)

print('Unique values of the target variable', unique)
print('Counts of the target variable :', counts)

In [ ]:
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers. 
df.iloc[:,4] = le.fit_transform(df.iloc[:,4]).astype('float64')
df.iloc[:,2] = le.fit_transform(df.iloc[:,2]).astype('float64')
df.iloc[:,3] = le.fit_transform(df.iloc[:,3]).astype('float64')
df.iloc[:,43] = le.fit_transform(df.iloc[:,43]).astype('float64')

df.isnull().sum()

In [ ]:
X_train = df.drop(['label', 'attack_cat', 'ï»¿id'], axis = 1)

y_train = df['attack_cat']

X_train.shape, y_train.shape

In [ ]:
df_test=pd.read_csv('../input/unsw-nb15/UNSW_NB15_testing-set.csv', encoding='latin')

df_test.head()

In [ ]:
df_test['attack_cat'].unique()

In [ ]:
(unique, counts) = np.unique(df_test['attack_cat'], return_counts=True)

print('Unique values of the target variable', unique)
print('Counts of the target variable :', counts)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
plt.rcParams.update({'font.size': 14})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=15)
ax.bar(unique, counts)
plt.show()

In [ ]:
df_test.replace(to_replace=["Analysis"], value="Attacks_1", inplace=True)
df_test.replace(to_replace=["Backdoor"], value="Attacks_1", inplace=True)
df_test.replace(to_replace=["Shellcode"], value="Attacks_1", inplace=True)
df_test.replace(to_replace=["Worms"], value="Attacks_1", inplace=True)

In [ ]:
df_test = df_test[df_test['attack_cat'] != 'Attacks_1']

In [ ]:
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers. 
df_test.iloc[:,4] = le.fit_transform(df_test.iloc[:,4]).astype('float64')
df_test.iloc[:,2] = le.fit_transform(df_test.iloc[:,2]).astype('float64')
df_test.iloc[:,3] = le.fit_transform(df_test.iloc[:,3]).astype('float64')
df_test.iloc[:,43] = le.fit_transform(df_test.iloc[:,43]).astype('float64')

df_test.isnull().sum()

In [ ]:
X_test = df_test.drop(['label', 'attack_cat', 'ï»¿id'], axis = 1)

y_test = df_test['attack_cat']

X_test.shape, y_test.shape

In [ ]:
bi_train_y = y_train.copy()
print(bi_train_y.unique())

bi_test_y = y_test.copy()
print(bi_test_y.unique())

### switch for binary!
y_train = bi_train_y
y_test = bi_test_y

In [ ]:
### transform ndarray back:
X_train = X_train.values
X_test = X_test.values

In [ ]:
### reshape input data to LSTM format [samples, time_steps, features]
X_train_lstm = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
print(f"shape of X_train:", X_train_lstm.shape)
print(f"shape of X_test:", X_test_lstm.shape)

In [ ]:
X_train_lstm = X_train_lstm[:, None, :]

In [ ]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [ ]:
def multiClassModel(N):
    model = Sequential()
    model.add(GRU(units=N, return_sequences=True, input_shape=(X_train.shape[1],1)))
    model.add(Dropout(0.2))
    # Second LSTM layer
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    # Third LSTM layer
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    # Fourth LSTM layer
    model.add(GRU(units=50))
    model.add(Dropout(0.2))
    # The output layer
    model.add(Dense(units=6, activation="softmax", name="softmax"))

    model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.summary()
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend

N= len(df.columns)
# from keras.callbacks import EarlyStopping
callback = EarlyStopping(patience=20, mode='min', restore_best_weights=True)
backend.clear_session()
model = multiClassModel(N)
history = model.fit(X_train,y_train,epochs=30,batch_size=64, callbacks=[callback])
### check the loss trend of epochs
pd.DataFrame(history.history).plot(kind='line', xlabel='epochs', figsize=(8, 6))

import matplotlib.pyplot as plt
plt.show()

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
# predicting on training set
y_train_pred_prob = model.predict(X_train)
y_test_pred_prob = model.predict(X_test)
y_train_pred = np.argmax(y_train_pred_prob, axis=1)
y_test_pred = np.argmax(y_test_pred_prob, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

labels=le.classes_

cm = confusion_matrix(y_train, y_train_pred)

fig, ax = plt.subplots(figsize=(8, 8))
plt.rcParams.update({'font.size': 15})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
#ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_).plot(ax=ax)
ax= plt.subplot();
sns.heatmap(cm, annot=True, fmt='g', ax=ax, xticklabels=labels, yticklabels=labels, cmap='Blues')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
#plt.show()

In [ ]:
labels=le.classes_

cm = confusion_matrix(y_test, y_test_pred)

fig, ax = plt.subplots(figsize=(8, 8))
plt.rcParams.update({'font.size': 15})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
#ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_).plot(ax=ax)
ax= plt.subplot();
sns.heatmap(cm, annot=True, fmt='g', ax=ax, xticklabels=labels, yticklabels=labels, cmap='Blues')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
#plt.show()

In [ ]:
def multilabel_matrix(y_true, y_pred, labels=None):
    mlm = multilabel_confusion_matrix(y_true, y_pred, labels=labels)
    df_performance = pd.DataFrame(index=labels, columns=['accuracy', 'precision', 'recall', 'f1_score', 'FPR'])
    for i, label in enumerate(labels):
        tn, fp, fn, tp = mlm[i].ravel()
        accuracy = (tn + tp) / (tn + fp + fn + tp)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        fpr = fp / (tn + fp)

        f1_score = 2*precision * recall / (precision + recall)
        df_performance.loc[label] = [round(accuracy, 4), round(precision,4), \
                                     round(recall, 4), round(f1_score,4), round(fpr,4)]
    return df_performance

In [ ]:
def FPR(y_true, y_pred, labels=None):
    mlm = multilabel_confusion_matrix(y_true, y_pred, labels=labels)
    df_performance = pd.DataFrame(index=labels, columns=['FPR'])
    for i, label in enumerate(labels):
        tn, fp, fn, tp = mlm[i].ravel()
        specificity = (tn) / (tn + fp)
        
        df_performance.loc[label] = 1 - specificity
    
    return df_performance

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from itertools import cycle
def RoC_Curve(y_score, y, labels, title): 
    y_cat = to_categorical(y)
    
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    lw = 2
    # First aggregate all false positive rates
    n_classes = len(labels)
#     print('n_classes:', n_classes)

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_cat[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_cat.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(figsize=(8, 8))
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.4f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.4f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)
           
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=lw,
                 label=f'ROC curve of class {labels[i]} (area = {roc_auc[i]:0.4f})')

       
    plt.rcParams.update({'font.size': 11})    
    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([-0.1, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title, fontsize=16)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
y_no = np.unique(y_train)
print(y_no)

In [ ]:
le.classes_

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

print(classification_report(y_test, y_test_pred))

performance = multilabel_matrix(y_test, y_test_pred, labels=y_no)
performance['Attack'] = le.classes_

performance

In [ ]:
performance = FPR(y_test, y_test_pred, labels=y_no)
performance['Attack'] = le.classes_

performance

In [ ]:
print(np.unique(y_train, return_counts=True))

In [ ]:
y_train = np.unique(y_train[0])

In [ ]:
RoC_Curve(y_test_pred_prob, y_test, le.classes_, title='ROC for test set')

In [ ]:
def GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=np.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=np.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=np.zeros(dim)
    Delta_score=float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim
    
    #Initialize the positions of search agents
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = np.random.uniform(0,1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
    
    Convergence_curve=np.zeros(Max_iter)

     # Loop counter
    print("GWO is optimizing  \""+objf.__name__+"\"")    

    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i,j]=np.clip(Positions[i,j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
        
        a=2-l*((2)/Max_iter); # a decreases linearly fron 2 to 0
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2       
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)        
        Convergence_curve[l]=Alpha_score;

        #if (l%1==0):
               #print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
    
    print(Positions.shape)
    print("Alpha position=",Alpha_pos);
    print("Beta position=",Beta_pos);
    print("Delta position=",Delta_pos);
    return Alpha_pos,Beta_pos;

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import sys
sys.path.append('/kaggle/input/benchmarks/')
import benchmarks

In [ ]:
import random

iters=1000
wolves=200
dimension=30
search_domain=[0,1]
lb=-5.28
ub=5.28
colneeded=[0,1,2,4,5,7,8,10,11,13,14]
modified_data=pd.DataFrame()
for i in colneeded:
    modified_data[df.columns[i]]=df[df.columns[i]].astype(float)
func_details=benchmarks.getFunctionDetails(6)

for i in range(0,10):
    alpha,beta=GWO(getattr(benchmarks,'F7'),lb,ub,dimension,wolves,iters)

In [ ]:
##Applying feature selection on the given dataset
##considering alpha as best solution and putting a threshold
threshold=-0.9
index=[]
print("alpha shape=",alpha.shape[0])
modified_daata=pd.DataFrame();
modified_daata_test=pd.DataFrame();
for i in range(0,alpha.shape[0]):
    if(alpha[i]>=threshold):
        modified_daata[df.columns[i]]=df[df.columns[i]].astype(float)
        modified_daata_test[df_test.columns[i]]=df_test[df_test.columns[i]].astype(float)
print("The modified data is following")
modified_daata.head()

In [ ]:
modified_daata_test.head()

In [ ]:
X_train = modified_daata
y_train = df['attack_cat'].values

X_test = modified_daata_test
y_test = df_test['attack_cat'].values

#X_train, X_test, y_train, y_test = train_test_split(modified_daata, Y, test_size = 0.40, random_state = 14)
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)
#print(X_train[0])
modified_daata.info()
modified_daata_test.info()

In [ ]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend

N= len(modified_daata.columns)
# from keras.callbacks import EarlyStopping
callback = EarlyStopping(patience=20, mode='min', restore_best_weights=True)
backend.clear_session()
model = multiClassModel(N)
history = model.fit(X_train,y_train,epochs=30,batch_size=64, callbacks=[callback])
### check the loss trend of epochs
pd.DataFrame(history.history).plot(kind='line', xlabel='epochs', figsize=(8, 6))

import matplotlib.pyplot as plt
plt.show()

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
# predicting on training set
y_train_pred_prob = model.predict(X_train)
y_test_pred_prob = model.predict(X_test)
y_train_pred = np.argmax(y_train_pred_prob, axis=1)
y_test_pred = np.argmax(y_test_pred_prob, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

labels=le.classes_

cm = confusion_matrix(y_train, y_train_pred)

fig, ax = plt.subplots(figsize=(5, 5))
plt.rcParams.update({'font.size': 15})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
#ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_).plot(ax=ax)
ax= plt.subplot();
sns.heatmap(cm, annot=True, fmt='g', ax=ax, xticklabels=labels, yticklabels=labels, cmap='Blues')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
#plt.show()

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

fig, ax = plt.subplots(figsize=(5, 5))
plt.rcParams.update({'font.size': 15})
ax.set_xlabel('Time [s]', fontsize='large', fontweight='bold')
ax.set_ylabel('Time [s]', fontsize='large', fontweight='bold')
#ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_).plot(ax=ax)
ax= plt.subplot();
sns.heatmap(cm, annot=True, fmt='g', ax=ax, xticklabels=labels, yticklabels=labels, cmap='Blues')
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
#plt.show()

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

print(classification_report(y_test, y_test_pred))
#performance = multilabel_matrix(y_train_pred, y_train, labels=y_test)
#performance

In [ ]:
print(np.unique(y_train, return_counts=True))

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

print(classification_report(y_test, y_test_pred))

performance = multilabel_matrix(y_test, y_test_pred, labels=y_no)
performance['Attack'] = le.classes_

performance

In [ ]:
RoC_Curve(y_train_pred_prob, y_train, le.classes_, title='ROC for UNSW-NB15 training set')

In [ ]:
RoC_Curve(y_test_pred_prob, y_test, le.classes_, title='ROC for UNSW-NB15 test set')